# install cool stuff

In [1]:
!pip install geopandas

# import stuff

In [3]:
import json
from shapely.geometry import Polygon

# Load the points geo json data as a geo dataframe

In [22]:
import plotly.figure_factory as ff
import plotly.express as px
import geopandas

filename = "dk_pop_100m"

points_df = geopandas.read_file(f'../geojson_files/{filename}_points.geojson')

points_df

,data,geometry
0,1.206668,POINT (10.60083 57.74333)
1,0.982903,POINT (10.60167 57.74333)
2,0.970523,POINT (10.63333 57.74333)
3,1.223284,POINT (10.63083 57.74083)
4,1.048370,POINT (10.59583 57.74000)
...,...,...
846588,0.870005,POINT (11.94083 54.56917)
846589,0.815470,POINT (11.95167 54.56917)
846590,0.536970,POINT (11.97250 54.56833)
846591,0.565375,POINT (11.95167 54.56583)


In [23]:
len(points_df.index)

846593

In [24]:
points_df['lon'] = points_df['geometry'].x
points_df['lat'] = points_df['geometry'].y

points_df

,data,geometry,lon,lat
0,1.206668,POINT (10.60083 57.74333),10.600833,57.743333
1,0.982903,POINT (10.60167 57.74333),10.601667,57.743333
2,0.970523,POINT (10.63333 57.74333),10.633333,57.743333
3,1.223284,POINT (10.63083 57.74083),10.630833,57.740833
4,1.048370,POINT (10.59583 57.74000),10.595833,57.740000
...,...,...,...,...
846588,0.870005,POINT (11.94083 54.56917),11.940833,54.569167
846589,0.815470,POINT (11.95167 54.56917),11.951667,54.569167
846590,0.536970,POINT (11.97250 54.56833),11.972500,54.568333
846591,0.565375,POINT (11.95167 54.56583),11.951667,54.565833


In [25]:
records = points_df[["lon", "lat"]].to_records(index=False)
lst = list(records)

#for e in list(map(lambda x: x[1], lst)):
#    print(e)

### The current polygons are squares, so we calculate the distance between points and set that as the height and widht of the polygons

In [26]:
curr = lst[0]

dct = {}

#go through all points and accumulate all unique distances between adjacent points in latitude (y axis but geodata)
# either go through all elemnts or max 5000

for i in range(1, min(len(lst),5000)):
    next = lst[i]
    
    if next[1] != curr[1]: # new line
        diff = curr[1] - next[1]
        if diff not in dct:
            dct[diff] = 1
        else: 
            dct[diff] = dct[diff] + 1
    
    curr = next
    
#create a dictionary of unique height differences

In [27]:
# set height and width of the polygons to be the biggest size difference between all points
max_diff = max(dct, key=dct.get)

max_diff

0.0008333333301138168

In [28]:
height = max_diff
width = max_diff

In [29]:
poly_df = geopandas.GeoDataFrame({})
def createPolygon(row):
    # create polygon(square) around points by using width and height
    points = [
        (row.lon - width/2, row.lat - height/2),
        (row.lon + width/2, row.lat - height/2),
        (row.lon + width/2, row.lat + height/2),
        (row.lon - width/2, row.lat + height/2),
        (row.lon - width/2, row.lat - height/2)
    ]
    return Polygon(points)



poly_df["geometry"] = points_df.apply(lambda row: createPolygon(row), axis = 1)
poly_df["population_density"] = points_df["data"]
poly_df["lon"] = points_df["lon"]
poly_df["lat"] = points_df["lat"]
poly_df["id"] = [i for i in range(len(points_df.index))]

poly_df.crs = points_df.crs

poly_df

,geometry,population_density,lon,lat,id
0,"POLYGON ((10.60042 57.74292, 10.60125 57.74292...",1.206668,10.600833,57.743333,0
1,"POLYGON ((10.60125 57.74292, 10.60208 57.74292...",0.982903,10.601667,57.743333,1
2,"POLYGON ((10.63292 57.74292, 10.63375 57.74292...",0.970523,10.633333,57.743333,2
3,"POLYGON ((10.63042 57.74042, 10.63125 57.74042...",1.223284,10.630833,57.740833,3
4,"POLYGON ((10.59542 57.73958, 10.59625 57.73958...",1.048370,10.595833,57.740000,4
...,...,...,...,...,...
846588,"POLYGON ((11.94042 54.56875, 11.94125 54.56875...",0.870005,11.940833,54.569167,846588
846589,"POLYGON ((11.95125 54.56875, 11.95208 54.56875...",0.815470,11.951667,54.569167,846589
846590,"POLYGON ((11.97208 54.56792, 11.97292 54.56792...",0.536970,11.972500,54.568333,846590
846591,"POLYGON ((11.95125 54.56542, 11.95208 54.56542...",0.565375,11.951667,54.565833,846591


In [30]:
poly_df.to_file(f'../geojson_files/{filename}_polygons.geojson', driver="GeoJSON")
#poly_df.to_csv("dk_polygons_1k.csv")